In [1]:
import ee
import geemap
import pandas as pd
from sklearn import ensemble
from geemap import ml
import json
import geopandas as gpd
from geemap import geojson_to_ee
from geemap.conversion import ee_to_geojson

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AbUR2VNo3TYkgQw1LzSekyAE-RaOaQl5czpQXz2VKxA8FvzdblgoujGNVjM



Successfully saved authorization token.


In [14]:
# 创建一个指定内罗毕范围的Geometry对象，这里使用一个2km x 2km的范围
nairobi_geom = ee.Geometry.Polygon(
[ [
[31.233587, 30.059156],
[31.233587, 30.084036],
[31.258305, 30.084036],
[31.258305, 30.059156],
[31.233587, 30.059156]
]])

# 从LANDSAT获取待处理图像
imagecol = ee.ImageCollection('LANDSAT/LC08/C01/T1')
image = ee.Algorithms.Landsat.simpleComposite(
    collection=imagecol.filterDate('2021-01-01', '2021-12-31'), asFloat=True)

In [18]:
# 加载训练数据
url = "https://raw.githubusercontent.com/giswqs/geemap/master/examples/data/rf_example.csv"
df = pd.read_csv(url)

# 选择独立变量和输出变量
X = df[['B2', 'B3', 'B4', 'B5', 'B6', 'B7']]
y = df['landcover']

# 创建一个Feature，使用手动输入的nairobi_geom作为几何体
nairobi_zone = ee.Feature(nairobi_geom)

# 选择机器学习算法并拟合
model_rf = ensemble.RandomForestClassifier(n_estimators=10).fit(X, y)

# 使用geemap将数学模型转换为字符串
trees = ml.rf_to_strings(model_rf, ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])

# 将字符串转换为EE分类器
ee_classifier = ml.strings_to_classifier(trees)

# 使用原始波段对图像进行分类
classified = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']).classify(ee_classifier)

# 裁剪图像
image = image.clip(nairobi_geom)
classified = classified.clip(nairobi_geom)

# 定义类别标签和名称
labels = {
    0: 'Building',
    1: 'Forest',
    2: 'Water',
}

# 重新分类图像
unclassified_class = 0
classified_urban = classified.remap([1, 2], [1, 2], unclassified_class).rename('classification')  # 将未分类的像素值设置为unclassified_class

# 创建地图
Map = geemap.Map(center=[-1.2921, 36.8219], zoom=14)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.5}, 'Nairobi Landsat')

# 自定义调色板
palette = ['#0000FF', '#FF0000', '#00FF00']

# 将调色板应用于分类图像
colored_classified = classified.visualize(palette=palette)
Map.addLayer(colored_classified, {}, 'Nairobi Classified Colored')

# 显示地图
Map

# 计算NDVI
ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
Map.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI')

# 计算NDBI
ndbi = image.normalizedDifference(['B6', 'B5']).rename('NDBI')
Map.addLayer(ndbi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDBI')

# 计算指定范围内各个类别的面积、比例和NDVI、NDBI值
def calculate_class_area_and_ndvi_ndbi(zone, classified_image, ndvi_image, ndbi_image):
    total_area = zone.geometry().area().getInfo()
    class_area = {}
    for label, class_name in labels.items():
        stats = classified_image.eq(label).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=zone.geometry(),
            scale=30,
            maxPixels=1e9
        )
        area = stats.get('classification').getInfo() * 900  # 乘以像素尺寸 (30m x 30m)
        class_area[f'{class_name}_area'] = area
        class_area[f'{class_name}_percentage'] = (area / total_area) * 100

    # 计算区域内的平均NDVI值
    ndvi_stats = ndvi_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=zone.geometry(),
        scale=30,
        maxPixels=1e9
    )
    class_area['NDVI_mean'] = ndvi_stats.get('NDVI').getInfo()

    # 计算区域内的平均NDBI值
    ndbi_stats = ndbi_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=zone.geometry(),
        scale=30,
        maxPixels=1e9
    )
    class_area['NDBI_mean'] = ndbi_stats.get('NDBI').getInfo()

    return zone.set(class_area)

zone_info = calculate_class_area_and_ndvi_ndbi(nairobi_zone, classified, ndvi, ndbi)

# 将计算结果导出为CSV文件
output_file = r'E:\Master Works\Conflict Urbanism\result.csv'
selectors = ['system:index', 'NDVI_mean', 'NDBI_mean']
for label, class_name in labels.items():
    selectors.extend([f'{class_name}_area', f'{class_name}_percentage'])
geemap.ee_export_vector(ee.FeatureCollection([zone_info]), output_file, selectors=selectors)

print("CSV文件已导出到：", output_file)

# 定义NDVI输出文件名和路径
output_ndvi_image = r'E:\Master Works\Conflict Urbanism\result_ndvi.tif'

# 将NDVI图像导出为GeoTIFF
geemap.ee_export_image(ndvi, filename=output_ndvi_image, scale=30, region=nairobi_geom, file_per_band=False)

print("NDVI图像已导出到：", output_ndvi_image)

# 定义NDBI输出文件名和路径
output_ndbi_image = r'E:\Master Works\Conflict Urbanism\result_ndbi.tif'

# 将NDBI图像导出为GeoTIFF
geemap.ee_export_image(ndbi, filename=output_ndbi_image, scale=30, region=nairobi_geom, file_per_band=False)

print("NDBI图像已导出到：", output_ndbi_image)

# 定义分类输出文件名和路径
output_classified_image = r'E:\Master Works\Conflict Urbanism\result_classified.tif'

# 将分类图像导出为GeoTIFF
geemap.ee_export_image(classified_urban, filename=output_classified_image, scale=30, region=nairobi_geom, file_per_band=False)

print("分类图像已导出到：", output_classified_image)


Generating URL ...
Please wait ...
Data downloaded to E:\Master Works\Conflict Urbanism\result.csv
CSV文件已导出到： E:\Master Works\Conflict Urbanism\result.csv
Generating URL ...
Please wait ...
Data downloaded to E:\Master Works\Conflict Urbanism\result_ndvi.tif
NDVI图像已导出到： E:\Master Works\Conflict Urbanism\result_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Master Works\Conflict Urbanism\result_ndbi.tif
NDBI图像已导出到： E:\Master Works\Conflict Urbanism\result_ndbi.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Master Works\Conflict Urbanism\result_classified.tif
分类图像已导出到： E:\Master Works\Conflict Urbanism\result_classified.tif
